### Research Question:
How does the distribution of voting power in DAOs impact the success of proposals and the formation of influence networks?

## Top 10 DAOs by Votes

1. **Stargate Finance** - 16.6M votes  
2. **Arbitrum** - 5.9M votes  
3. **Magic Square** - 5.5M votes  
4. **Optimism** - 1.9M votes  
5. **Aave** - 1.7M votes  
6. **Aavegotchi DAO** - 420.1k votes  
7. **PancakeSwap** - 311.6k votes  
8. **GMX** - 292.5k votes  
9. **Uniswap** - 277.7k votes  
10. **Metis DAO** - 262.4k votes  

Source: https://deepdao.io/organizations


- We retrieve 100 proposal for each space
- We retrieve top 1000 voters (by voting power) for each proposal


#### Bi-partite Network
##### Nodes:
- Proposal nodes: Represent governance decisions.
- Voter nodes: Represent participants with associated voting power.
##### Edges:
- Connect voters to proposals, weighted by their voting power.

##### Project onto voters
Projection creates a unipartite network by connecting nodes of the same set (e.g., voters) based on shared connections to the other set (e.g., proposals).
 By projecting onto voters, we create edges between voters who voted on the same proposals, revealing patterns of shared behavior or preferences.
 
 Example: If Voter A and Voter B both voted on Proposal X, they are connected in the projected network.

#### Bipartite Network:
Use it when:
- Analyzing voter-to-proposal relationships.
- Studying the structure and distribution of participation across proposals.
- Exploring the role of proposals in connecting different voters.
#### Projected Voter Network:
Use it when:
- Focusing on voter-to-voter relationships.
- Studying collaboration, alignment, or community formation among voters.
- Simplifying the graph to analyze the social dynamics of voters.

In [2]:
import time
import requests
import networkx as nx

# Snapshot API URL
SNAPSHOT_API_URL = "https://hub.snapshot.org/graphql"

# List of top 10 spaces
spaces_list = [
    'stgdao.eth', 'arbitrumfoundation.eth', 'magicappstore.eth',
    'opcollective.eth', 'aave.eth', 'aavegotchi.eth',
    'cakevote.eth', 'gmx.eth', 'uniswapgovernance.eth', 'metislayer2.eth'
]


In [3]:

# Function to handle rate-limited requests
def fetch_with_rate_limiting(query):
    while True:
        response = requests.post(SNAPSHOT_API_URL, json={"query": query})
        if response.status_code == 429:  # Too many requests
            print("Rate limit hit. Waiting 60 seconds...")
            time.sleep(60)  # Wait and retry
        elif response.status_code == 200:
            return response.json()
        else:
            print(f"Unexpected error: {response.status_code}")
            time.sleep(10)  # Retry after short delay

# Fetch top proposals for a space
def fetch_top_proposals(space_id, batch_size=5):
    query = f"""
    query {{
      proposals(first: {batch_size}, where: {{ space: "{space_id}" }}, orderDirection: desc) {{
        id
        title
        space {{
          id
          name
        }}
      }}
    }}
    """
    data = fetch_with_rate_limiting(query)
    return data['data']['proposals']

# Fetch voters for a proposal and sort by voting power
def fetch_top_voters(proposal_id, batch_size=1000):
    query = f"""
    query {{
      votes(
        first: {batch_size},
        where: {{ proposal: "{proposal_id}" }},
        orderBy: "vp",
        orderDirection: desc
      ) {{
        voter
        vp  # Voting power
      }}
    }}
    """
    data = fetch_with_rate_limiting(query)
    votes = data['data']['votes']
    # Sort voters by voting power in descending order
    votes_sorted = sorted(votes, key=lambda x: x['vp'], reverse=True)
    return votes_sorted

# Construct a bipartite network
def construct_network(spaces):
    B = nx.Graph()
    for space_id in spaces:
        print(f"Fetching top proposals for space: {space_id}")
        proposals = fetch_top_proposals(space_id, batch_size=100)

        for proposal in proposals:
            proposal_id = proposal['id']
            space_name = proposal['space']['name']

            # Add proposal node
            B.add_node(proposal_id, type='proposal', title=proposal['title'], space=space_id, space_name=space_name)

            print(f"  Fetching top voters for proposal: {proposal['title']} (ID: {proposal_id})")
            voters = fetch_top_voters(proposal_id, batch_size=1000)

            for voter in voters:
                voter_id = voter['voter']
                vp = voter['vp']  # Voting power

                # Add voter node with voting power attribute
                B.add_node(voter_id, type='voter', voting_power=vp)

                # Add edge between voter and proposal
                B.add_edge(voter_id, proposal_id, weight=vp)  # Edge weight as voting power

    return B

# Main execution
print("Constructing the network...")
bipartite_network = construct_network(spaces_list)
print(f"Number of nodes: {bipartite_network.number_of_nodes()}")
print(f"Number of edges: {bipartite_network.number_of_edges()}")
nx.write_gexf(bipartite_network, "bipartite_network.gexf")

# Project onto voter network
voters = [n for n, d in bipartite_network.nodes(data=True) if d['type'] == 'voter']
voter_network = nx.algorithms.bipartite.projected_graph(bipartite_network, voters)
print(f"Projected voter network density: {nx.density(voter_network)}")

# Save or analyze network further
nx.write_gexf(voter_network, "voter_network.gexf")

Constructing the network...
Fetching top proposals for space: stgdao.eth
  Fetching top voters for proposal: Stargate Deployment on Hedera (ID: 0x1007f3e7e228cb432a861565a127da4d8c4a5476dddbb0eb088e44b4ea9c2449)
  Fetching top voters for proposal: ETH Pool on Manta Pacific (ID: 0xc51bf1e91c1e0de4d84395818722d402a0c989f07f72e7ff5060bf16124b4b2f)
  Fetching top voters for proposal: Stargate Deployment on Conflux (ID: 0x45de70585d306879ea79de956ab5d136acd03b97d458ef03f09d61a05f1bf91f)
  Fetching top voters for proposal: Final Data RFP Vote - Choice of Artemis Options (ID: 0x756e42595e5d9599dde8a9c1e01102cfe7ec6e8dfb1732320236b5f30a049a14)
  Fetching top voters for proposal: Stargate Deployment to Gnosis Chain (ID: 0x9aa8b42775462de79cebe8a376059056745f72a91486d518d57a47e5a78a78ed)
  Fetching top voters for proposal: Absolutely thrilling news for all Stargate users! (ID: 0x71124af7d657d83117ebc87349c342c7d6eef062027c246f354cdc027f579900)
  Fetching top voters for proposal: Absolutely thril

KeyboardInterrupt: 